# scraping

#### scraper <-> crawler
- scraper: 페이지에서 필요한 내용을 찾아 가져오는 것.

- crawler: anchor 태그를 찾아 링크를 타고 돌아다니는 것.

#### 사이트가 표준을 잘 지켰는지 검사해주는 사이트
- https://validator.w3.org/unicorn/?ucn_lang=ko
- 여기서 http://ppomppu.co.kr 을 검색해보면 얼마나 거지같이 만들었는지 알 수 있다.

#### 뽐뿌는 국내 3대 커뮤니티로 큰 규모 사이트이면서 거지같이 만든 사이트의 대표격임.
- 극악의 뽐뿌를 크로울링 해보면 왠만한 사이트는 크로울링할 수 있게 될 것이다.

----

# 뽐뿌에서 (제목, 추천수, 조회수)를 리스트로 크로울링하기

In [2]:
import requests
import time

def download(method, url, params=None, data=None, headers=None, timeout=1, retries=4):
    if headers == None:
        headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}
    
    try:
        resp = requests.request(method, url, 
                                params=params, data=data, 
                                headers=headers, timeout=timeout)
        resp.raise_for_status()
        
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and retries > 0:
            print("remained retry:", retries)
            time.sleep(timeout)
            resp = download(method, url, params, data, headers, timeout, retries-1) 
        else:
            print(resp.status_code)
            print(resp.reason)
        
    return resp

In [3]:
from bs4 import BeautifulSoup

In [13]:
# GET방식 파라미터가 url에 안보여서 request 헤더 가져옴

url = "http://ppompu.co.kr"
params = {
    "ai": "C_CM2lGEtXcrVH8vmgQOo8KjoDo2qh7tX-o-PwuUJwI23ARABIP-epiFgm4PphJwpoAGaiJzdAsgBAqkCjXTFtF4aDT6oAwHIA5kEqgSxAU_Qw9BhZdOwvT2TcacvMh-RjgF5ZPdoQZBH5RMhEmCLpFxvNkKZh37HNcmagSb6OlzkIgB8tB7gBP8XrY1fnqwBEsLTnBe7NgGztaYSedKt8Agf87R0Hd53v2V-MK0qCsSL6Sq2AuPOyMyxas4GzuragitmrvrbNKGD3yKJDfyGCD7Z-24W76RFWMsDhan7uwNfKQ684nyf1RySFFd_OV9VwUMrk3LIALfVOGPv-FxYr-AEA5AGAaAGAoAHzvfjogGIBwGQBwKoB47OG6gH1ckbqAfB0xuoB4XUG6gHgdQbqAeC1BuoB4bUG6gHhNQbqAfg0xuoB7oGqAfZyxuoB8_MG6gHpr4b2AcA0ggGCAAQAhgNgAoDsBOo5vgG0BMA2BMD2BQB",
"sigh": "VwbL0y_4HsU",
"cid": "CAQSQwDwy9IZGsUXixiO1W-MIPAfjc9LYcKA4QkBOou3XTnwoJY6D5tfHQkbgytoUkGB5p0kvCKo1GVHlKzGJ1UGaEnDHL4",
"label": "window_focus",
"qqid": "CMrv0bHbuOMCFUtzYAodKDgK7Q",
"fg": "1"
}


In [14]:
html = download("get", url, params)

In [15]:
# 안됨 ㅜㅜ
# url 주소가 pp가 두개씩 있어야한다!
# ppomppu.co.kr 은 get방식 url이 안보임!! 

html.text

'<html>\n<head>\n<meta http-equiv="content-type" content="text/html; charset=utf-8">\n<title>ì\x82¬ë\x9e\x8cì\x9d´ ì¢\x8bì\x95\x84 í\x95¨ê»\x98í\x95\x98ë\x8a\x94ê³³.. ë½\x90ë¿\x8c!</title>\n<script type="text/javascript" src="http://code.jquery.com/jquery-1.10.2.min.js"></script>\n<script type="text/javascript" src="popup/jquery.cookie.js"></script>\n<script type="text/javascript" src="popup/popup_cfg.js"></script>\n<script type="text/javascript" src="popup/popup.js"></script>\n</head>\n<body style="margin:0;padding:0;overflow:hidden;">\n\t<iframe frameborder="0" width="100%" height="100%" src="http://www.ppomppu.co.kr/"></iframe>\n</body>\n</html> '

In [16]:
# 종익이한테 주소 받아옴

url = "http://www.ppomppu.co.kr/zboard/zboard.php"
html = download("get", url, params={"id":"ppomppu"})

# 뽐뿌는 well-formed가 아니니까 html.parser
dom = BeautifulSoup(html.text, "html.parser")

In [17]:
dom.select("font.list_title")

[<font class="list_title">[Gsuite] Gsuite Basic India (3700원(월) / 무료)</font>,
 <font class="list_title">[하이마트몰] 갤럭시버즈 (138,500원 / 무배) 오너스 첫달 무료체험시, 최대 50% 페이백</font>,
 <font class="list_title">[옥션] 식사에반하다25개입 (26,530원 / 무배)</font>,
 <font class="list_title">[티몬] (티몬페이전용)해피머니 10만원권 3장(271,500원/쿠폰적용시 269,900원)</font>,
 <font class="list_title">[티몬] 해피머니 10만원권 (90,900원/0원)</font>,
 <font class="list_title">[티몬] 1등급 한돈 삼겹살 600g (6,500/무료배송)</font>,
 <font class="list_title">[우체국쇼핑] 진안 매콤화끈 중독의맛 뼈없는 직화불닭발 1+1(2팩)(8500/무료) 쿠폰적용시7650</font>,
 <font class="list_title">[유튜브] 유튜브프리미엄 인도 ( 2200원(월), 3200원(월)6명 / 무료)</font>,
 <font class="list_title">[11번가] 벨킨 스마트폰 전기종 호환 충전케이블(9,900/무배)</font>,
 <font class="list_title">[네이버스토어팜] 뉴발란스 잔테v3 (49,800원/무료)</font>,
 <font class="list_title">[LG생활건강] 홈스타 뿌리는 곰팡이 (9,900/무료)</font>,
 <font class="list_title">[Gsshop] 본이트츄잉캔디(12900/무배)</font>,
 <font class="list_title">[11번가] LG 식기세척기 DFB22W (1,119,000/무료배송)</font>,
 <font class="list_title">[옥션] 유니온 135c

In [27]:
dom.select("tr.list0")[0].find_all({"font"})[0].text

'[하이마트몰] 갤럭시버즈 (138,500원 / 무배) 오너스 첫달 무료체험시, 최대 50% 페이백'

In [25]:
dom.select("tr.list0")[0].find_all("td", {"class":"eng list_vspace"})[-2:]

[<td class="eng list_vspace" colspan="2" nowrap="">9 - 5</td>,
 <td class="eng list_vspace" colspan="2" nowrap="">5803</td>]

In [35]:
want_list0 = [(tag.find_all("font")[0].text, 
  tag.find_all("td", {"class":"eng list_vspace"})[-2].text, 
  tag.find_all("td", {"class":"eng list_vspace"})[-1].text) for tag in dom.select("tr.list0")]
want_list0

[('[하이마트몰] 갤럭시버즈 (138,500원 / 무배) 오너스 첫달 무료체험시, 최대 50% 페이백', '9 - 5', '5803'),
 ('[옥션](스마일클럽전용딜) 생활공작소 제습제 520ml*12개*2 ->24개 (14,280원/무료) 하나/NH농협/KB국민/비씨/현대/신한 12,300원',
  '9 - 0',
  '2149'),
 ('[티몬] 해피머니 10만원권 (90,900원/0원)', '78 - 0', '12147'),
 ('[우체국쇼핑] 진안 매콤화끈 중독의맛 뼈없는 직화불닭발 1+1(2팩)(8500/무료) 쿠폰적용시7650',
  '6 - 0',
  '3868'),
 ('[KW 주식회사 ] 휴대용 미니 선풍기 ( 5,900~8,900 / 무료)', '0 - 1', '2382'),
 ('[네이버스토어팜] 뉴발란스 잔테v3 (49,800원/무료)', '10 - 0', '9139'),
 ('[네이버쇼핑] 잘만 CNPS10X OPTIMA II 블랙 CPU쿨러 (19,900/2500)', '1 - 0', '3102'),
 ('[11번가] LG 식기세척기 DFB22W (1,119,000/무료배송)', '', '2388'),
 ('[G9] 토니모리 에너지24 맨즈 올인원 모이스처 에센스 1+1 (11,000/무료배송)', '14 - 0', '9478'),
 ('[티몬] 루앱 차량용 고속 무선 충전 자동센서 거치대(18,900/무배)', '', '4786')]

In [36]:
want_list1 = [(tag.find_all("font")[0].text, 
  tag.find_all("td", {"class":"eng list_vspace"})[-2].text, 
  tag.find_all("td", {"class":"eng list_vspace"})[-1].text) for tag in dom.select("tr.list1")]
want_list1

[('[Gsuite] Gsuite Basic India (3700원(월) / 무료)', '3 - 0', '3844'),
 ('[옥션] 식사에반하다25개입 (26,530원 / 무배)', '', '3798'),
 ('[티몬] (티몬페이전용)해피머니 10만원권 3장(271,500원/쿠폰적용시 269,900원)', '33 - 0', '8296'),
 ('[티몬] 1등급 한돈 삼겹살 600g (6,500/무료배송)', '1 - 0', '5109'),
 ('[유튜브] 유튜브프리미엄 인도 ( 2200원(월), 3200원(월)6명 / 무료)', '77 - 16', '35288'),
 ('[11번가] 벨킨 스마트폰 전기종 호환 충전케이블(9,900/무배)', '3 - 0', '3208'),
 ('[LG생활건강] 홈스타 뿌리는 곰팡이 (9,900/무료)', '3 - 0', '2175'),
 ('[Gsshop] 본이트츄잉캔디(12900/무배)', '', '1295'),
 ('[옥션] 유니온 135c 55인치 UHDTV  (299,000/무료)', '2 - 2', '5558'),
 ('[티몬] 초코무쵸 27g 40봉 (7,900원/무료)', '10 - 0', '2351')]

In [38]:
wants = want_list0 + want_list1
wants

[('[하이마트몰] 갤럭시버즈 (138,500원 / 무배) 오너스 첫달 무료체험시, 최대 50% 페이백', '9 - 5', '5803'),
 ('[옥션](스마일클럽전용딜) 생활공작소 제습제 520ml*12개*2 ->24개 (14,280원/무료) 하나/NH농협/KB국민/비씨/현대/신한 12,300원',
  '9 - 0',
  '2149'),
 ('[티몬] 해피머니 10만원권 (90,900원/0원)', '78 - 0', '12147'),
 ('[우체국쇼핑] 진안 매콤화끈 중독의맛 뼈없는 직화불닭발 1+1(2팩)(8500/무료) 쿠폰적용시7650',
  '6 - 0',
  '3868'),
 ('[KW 주식회사 ] 휴대용 미니 선풍기 ( 5,900~8,900 / 무료)', '0 - 1', '2382'),
 ('[네이버스토어팜] 뉴발란스 잔테v3 (49,800원/무료)', '10 - 0', '9139'),
 ('[네이버쇼핑] 잘만 CNPS10X OPTIMA II 블랙 CPU쿨러 (19,900/2500)', '1 - 0', '3102'),
 ('[11번가] LG 식기세척기 DFB22W (1,119,000/무료배송)', '', '2388'),
 ('[G9] 토니모리 에너지24 맨즈 올인원 모이스처 에센스 1+1 (11,000/무료배송)', '14 - 0', '9478'),
 ('[티몬] 루앱 차량용 고속 무선 충전 자동센서 거치대(18,900/무배)', '', '4786'),
 ('[Gsuite] Gsuite Basic India (3700원(월) / 무료)', '3 - 0', '3844'),
 ('[옥션] 식사에반하다25개입 (26,530원 / 무배)', '', '3798'),
 ('[티몬] (티몬페이전용)해피머니 10만원권 3장(271,500원/쿠폰적용시 269,900원)', '33 - 0', '8296'),
 ('[티몬] 1등급 한돈 삼겹살 600g (6,500/무료배송)', '1 - 0', '5109'),
 ('[유튜브] 유튜브프리미엄 인도 ( 2200원(월), 3

In [44]:
len(wants)

20

---

In [49]:
[ (tag.find_all("font")[0].text, 
  tag.find_all("td", {"class":"eng list_vspace"})[-2].text, 
  tag.find_all("td", {"class":"eng list_vspace"})[-1].text) 
 for tag in dom.select("tr.list0") + dom.select("tr.list1")]

[('[하이마트몰] 갤럭시버즈 (138,500원 / 무배) 오너스 첫달 무료체험시, 최대 50% 페이백', '9 - 5', '5803'),
 ('[옥션](스마일클럽전용딜) 생활공작소 제습제 520ml*12개*2 ->24개 (14,280원/무료) 하나/NH농협/KB국민/비씨/현대/신한 12,300원',
  '9 - 0',
  '2149'),
 ('[티몬] 해피머니 10만원권 (90,900원/0원)', '78 - 0', '12147'),
 ('[우체국쇼핑] 진안 매콤화끈 중독의맛 뼈없는 직화불닭발 1+1(2팩)(8500/무료) 쿠폰적용시7650',
  '6 - 0',
  '3868'),
 ('[KW 주식회사 ] 휴대용 미니 선풍기 ( 5,900~8,900 / 무료)', '0 - 1', '2382'),
 ('[네이버스토어팜] 뉴발란스 잔테v3 (49,800원/무료)', '10 - 0', '9139'),
 ('[네이버쇼핑] 잘만 CNPS10X OPTIMA II 블랙 CPU쿨러 (19,900/2500)', '1 - 0', '3102'),
 ('[11번가] LG 식기세척기 DFB22W (1,119,000/무료배송)', '', '2388'),
 ('[G9] 토니모리 에너지24 맨즈 올인원 모이스처 에센스 1+1 (11,000/무료배송)', '14 - 0', '9478'),
 ('[티몬] 루앱 차량용 고속 무선 충전 자동센서 거치대(18,900/무배)', '', '4786'),
 ('[Gsuite] Gsuite Basic India (3700원(월) / 무료)', '3 - 0', '3844'),
 ('[옥션] 식사에반하다25개입 (26,530원 / 무배)', '', '3798'),
 ('[티몬] (티몬페이전용)해피머니 10만원권 3장(271,500원/쿠폰적용시 269,900원)', '33 - 0', '8296'),
 ('[티몬] 1등급 한돈 삼겹살 600g (6,500/무료배송)', '1 - 0', '5109'),
 ('[유튜브] 유튜브프리미엄 인도 ( 2200원(월), 3

---

In [53]:
# 클래스 이름이 list로 시작하는 이라는 뜻  =>  ^=

dom.select("tr[class^=list]")

[<tr align="center" class="list_notice" id="page_show_noti_1" name="page_show_noti_1" onmouseout="this.style.backgroundColor=''" onmouseover="this.style.backgroundColor='#F5F5F5'" style="height:50;word-break:break-all;display:table-row;*display:block;" valign="middle">
 <td align="center" colspan="2">
 <img alt="notice" border="0" src="/images/notice_icon.jpg"/></td>
 <td class="han list_vspace" colspan="2" nowrap=""><nobr class="han list_vspace">기타</nobr></td>
 <!--<td nowrap colspan=2 style='padding:0'><input type=checkbox name=cart value="6"></td>-->
 <td align="left" class="list_vspace" colspan="2"><div class="list_name" style="width:80px;overflow:hidden;text-overflow:ellipsis"><nobr class="list_vspace"> <a href="#" onclick="return false"><img align="absmiddle" alt="관리자3" border="0" src="//nic.ppomppu.co.kr/zboard/nickcon/157163.gif?v=201907161441"/></a></nobr></div></td> <td align="left" class="list_vspace">
 <a href="view.php?id=regulation&amp;page=1&amp;divpage=202&amp;no=6">[필독

In [67]:
# 선생님 방법

for tr in dom.select("tr[class^=list]")[1:]:
    td = tr.find_all ("td", recursive=False)
    print(td[3].select("td[valign=middle] > a")[0].text)
    temp = td[5].text.split("-")
    print((0, 0) if len(temp) < 2 else ",".join(temp)) # 추천 비추천이 없는 게시물이 에러를 낼 수 있으므로 예외처리함.
    print(td[6].text.strip()) # strip은 공백 제거하는 메서드

[Gsuite] Gsuite Basic India (3700원(월) / 무료)
3 , 0
3844
[하이마트몰] 갤럭시버즈 (138,500원 / 무배) 오너스 첫달 무료체험시, 최대 50% 페이백
9 , 5
5803
[옥션] 식사에반하다25개입 (26,530원 / 무배)
(0, 0)
3798
[옥션](스마일클럽전용딜) 생활공작소 제습제 520ml*12개*2 ->24개 (14,280원/무료) 하나/NH농협/KB국민/비씨/현대/신한 12,300원
9 , 0
2149
[티몬] (티몬페이전용)해피머니 10만원권 3장(271,500원/쿠폰적용시 269,900원)
33 , 0
8296
[티몬] 해피머니 10만원권 (90,900원/0원)
78 , 0
12147
[티몬] 1등급 한돈 삼겹살 600g (6,500/무료배송)
1 , 0
5109
[우체국쇼핑] 진안 매콤화끈 중독의맛 뼈없는 직화불닭발 1+1(2팩)(8500/무료) 쿠폰적용시7650
6 , 0
3868
[유튜브] 유튜브프리미엄 인도 ( 2200원(월), 3200원(월)6명 / 무료)
77 , 16
35288
[KW 주식회사 ] 휴대용 미니 선풍기 ( 5,900~8,900 / 무료)
0 , 1
2382
[11번가] 벨킨 스마트폰 전기종 호환 충전케이블(9,900/무배)
3 , 0
3208
[네이버스토어팜] 뉴발란스 잔테v3 (49,800원/무료)
10 , 0
9139
[LG생활건강] 홈스타 뿌리는 곰팡이 (9,900/무료)
3 , 0
2175
[네이버쇼핑] 잘만 CNPS10X OPTIMA II 블랙 CPU쿨러 (19,900/2500)
1 , 0
3102
[Gsshop] 본이트츄잉캔디(12900/무배)
(0, 0)
1295
[11번가] LG 식기세척기 DFB22W (1,119,000/무료배송)
(0, 0)
2388
[옥션] 유니온 135c 55인치 UHDTV  (299,000/무료)
2 , 2
5558
[G9] 토니모리 에너지24 맨즈 올인원 모이스처 에센스 1+1 (11,000/무료배송)
14 , 0
9478
[티몬

In [62]:
dom.select("tr[class^=list]")[0].find_all("td", {"class":"eng list_vspace"})[-2:]

[<td class="eng list_vspace" colspan="2" nowrap="">0</td>,
 <td class="eng list_vspace" colspan="2" nowrap="">2424970</td>]

In [63]:
[ (tag.find_all("font")[0].text, 
  *[i.text for i in tag.find_all("td", {"class":"eng list_vspace"})[-2:]]) 
 for tag in dom.select("tr[class^=list]")[1:]]

[('[Gsuite] Gsuite Basic India (3700원(월) / 무료)', '3 - 0', '3844'),
 ('[하이마트몰] 갤럭시버즈 (138,500원 / 무배) 오너스 첫달 무료체험시, 최대 50% 페이백', '9 - 5', '5803'),
 ('[옥션] 식사에반하다25개입 (26,530원 / 무배)', '', '3798'),
 ('[옥션](스마일클럽전용딜) 생활공작소 제습제 520ml*12개*2 ->24개 (14,280원/무료) 하나/NH농협/KB국민/비씨/현대/신한 12,300원',
  '9 - 0',
  '2149'),
 ('[티몬] (티몬페이전용)해피머니 10만원권 3장(271,500원/쿠폰적용시 269,900원)', '33 - 0', '8296'),
 ('[티몬] 해피머니 10만원권 (90,900원/0원)', '78 - 0', '12147'),
 ('[티몬] 1등급 한돈 삼겹살 600g (6,500/무료배송)', '1 - 0', '5109'),
 ('[우체국쇼핑] 진안 매콤화끈 중독의맛 뼈없는 직화불닭발 1+1(2팩)(8500/무료) 쿠폰적용시7650',
  '6 - 0',
  '3868'),
 ('[유튜브] 유튜브프리미엄 인도 ( 2200원(월), 3200원(월)6명 / 무료)', '77 - 16', '35288'),
 ('[KW 주식회사 ] 휴대용 미니 선풍기 ( 5,900~8,900 / 무료)', '0 - 1', '2382'),
 ('[11번가] 벨킨 스마트폰 전기종 호환 충전케이블(9,900/무배)', '3 - 0', '3208'),
 ('[네이버스토어팜] 뉴발란스 잔테v3 (49,800원/무료)', '10 - 0', '9139'),
 ('[LG생활건강] 홈스타 뿌리는 곰팡이 (9,900/무료)', '3 - 0', '2175'),
 ('[네이버쇼핑] 잘만 CNPS10X OPTIMA II 블랙 CPU쿨러 (19,900/2500)', '1 - 0', '3102'),
 ('[Gsshop] 본이트츄잉캔디(12900/무배)', '', 

---

In [78]:
# 선생님 방법
# 썸네일도 가져오기

for tr in dom.select("tr[class^=list]")[1:]:
    td = tr.find_all ("td", recursive=False)
    print(td[3].select("td[valign=middle] > a")[0].text)
    temp = td[5].text.split("-")
    print((0, 0) if len(temp) < 2 else ",".join(temp)) # 추천 비추천이 없는 게시물이 에러를 낼 수 있으므로 예외처리함.
    print(td[6].text.strip()) # strip은 공백 제거하는 메서드
    print(td[3].select("img[class=thumb_border]"))

[Gsuite] Gsuite Basic India (3700원(월) / 무료)
3 , 0
3844
[<img align="absmiddle" border="0" class="thumb_border" height="50" onerror="this.src='//static.ppomppu.co.kr/www/img/noimage/noimage_60x50.jpg'" onfocus="blur()" src="//cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/6/small_323876.jpg?t=20190716134948" style="margin:0px 5px 0px 5px;" width="60"/>]
[하이마트몰] 갤럭시버즈 (138,500원 / 무배) 오너스 첫달 무료체험시, 최대 50% 페이백
9 , 5
5803
[<img align="absmiddle" border="0" class="thumb_border" height="50" onerror="this.src='//static.ppomppu.co.kr/www/img/noimage/noimage_60x50.jpg'" onfocus="blur()" src="//cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/5/small_323875.jpg?t=20190716133713" style="margin:0px 5px 0px 5px;" width="60"/>]
[옥션] 식사에반하다25개입 (26,530원 / 무배)
(0, 0)
3798
[<img align="absmiddle" border="0" class="thumb_border" height="50" onerror="this.src='//static.ppomppu.co.kr/www/img/noimage/noimage_60x50.jpg'" onfocus="blur()" src="//cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/4/small_323874.jpg?t=2019

---

# 과제

뽐뿌 자유게시판(1시간동안)


[Crawling]
1. Focused Crawling(10페이지 내에서)
2. 각 페이지 당 게시글(제목) 부분의 a 수직
3. 수집된 a로부터 Scrapping


[Scrapping]
1. 제목, 본문내용, 댓글 파싱
(댓글은 있거나 여러개 있음)

=========== Scraping이 잘 안될 거임 ========

In [230]:
url = "http://www.ppomppu.co.kr/zboard/zboard.php"
params = {"id":"freeboard"}
html = download('get', url, params)

In [231]:
dom = BeautifulSoup(html.text, "html.parser")

### 1. 게시판의 게시물들의 url을 가져와보자

In [95]:
dom.select(".list_title")[0].find_parent()

<a href="/zboard/view.php?id=issue&amp;no=160987" onclick="informAdClick('http://s.ppomppu.co.kr/inform_ad_click.php?p=aW5mb3JtX2FkX3B2X2NsaWNrfDEwMTI%3D')">
<font class="list_title"><img align="absmiddle" border="0" src="/images/icon_04.png" style="margin:0px;"/>  [알려드립니다] 정치자유게시판 정식 운영에 대해 안내드립니다.</font>
</a>

In [98]:
title_urls = [requests.compat.urljoin(url, tag.find_parent()["href"]) for tag in dom.select(".list_title")]
title_urls

['http://www.ppomppu.co.kr/zboard/view.php?id=issue&no=160987',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543080',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543079',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543078',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543077',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543076',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543075',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543074',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543073',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543072',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543071',
 'http://www.ppomppu.co.kr

### 2. 게시판의 다음 페이지로 넘어가기 기능을 만들어보자

In [112]:
# 페이지 하단의 게시판 페이지 정보 있는 태그.

dom.select("td#page_list")[0]

<td id="page_list">
<font class="han">
<font class="page_inert">1</font> <a href="/zboard/zboard.php?id=freeboard&amp;page=2&amp;divpage=1221" onfocus="blur()">2</a> <a href="/zboard/zboard.php?id=freeboard&amp;page=3&amp;divpage=1221" onfocus="blur()">3</a> <a href="/zboard/zboard.php?id=freeboard&amp;page=4&amp;divpage=1221" onfocus="blur()">4</a> <a href="/zboard/zboard.php?id=freeboard&amp;page=5&amp;divpage=1221" onfocus="blur()">5</a> <a href="/zboard/zboard.php?id=freeboard&amp;page=6&amp;divpage=1221" onfocus="blur()">6</a> <a href="/zboard/zboard.php?id=freeboard&amp;page=7&amp;divpage=1221" onfocus="blur()">7</a> <a href="/zboard/zboard.php?id=freeboard&amp;page=8&amp;divpage=1221" onfocus="blur()">8</a> <a href="/zboard/zboard.php?id=freeboard&amp;page=9&amp;divpage=1221" onfocus="blur()">9</a> <a href="/zboard/zboard.php?id=freeboard&amp;page=10&amp;divpage=1221" onfocus="blur()">10</a> <a class="page_next" href="/zboard/zboard.php?id=freeboard&amp;page=11&amp;divpage=1221"

In [121]:
# 2페이지의 url 뽑기

dom.select("td#page_list")[0].find_all("a", text="2")[0]["href"]

'/zboard/zboard.php?id=freeboard&page=2&divpage=1221'

In [123]:
# urljoin으로 url 완성

next_page = requests.compat.urljoin(url, dom.select("td#page_list")[0].find_all("a", text="2")[0]["href"])
next_page

'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=2&divpage=1221'

In [132]:
# 현재 페이지 넘버

dom.select(".page_inert")[0].text

'1'

In [228]:
# 다음 페이지로 넘어가 게시물 url을 따오는 함수로 만듦.

def go_to_next(dom):
    current_page = int(dom.select(".page_inert")[0].text)
    next_url = requests.compat.urljoin(url, dom.select("td#page_list")[0].find_all("a", text=current_page+1)[0]["href"])
    next_html = download("get", next_url)
    next_dom = BeautifulSoup(next_html.text, "html.parser")        
    return [requests.compat.urljoin(url, tag.find_parent()["href"]) for tag in next_dom.select(".list_title")]

In [232]:
go_to_next(dom)

['http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=2&divpage=1221&no=6543294',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=2&divpage=1221&no=6543293',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=2&divpage=1221&no=6543292',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=2&divpage=1221&no=6543291',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=2&divpage=1221&no=6543290',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=2&divpage=1221&no=6543289',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=2&divpage=1221&no=6543288',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=2&divpage=1221&no=6543287',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=2&divpage=1221&no=6543285',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=2&divpage=1221&no=6543284',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=2&divpage=1221&no=6543283',

### 크로울링 중간 점검

In [144]:
# 가즈아~~

queue = []
queue.extend(title_urls)
seen = []

while queue:
    baseURL = queue.pop(0)
    seen.append(baseURL)
    
    # 게시물 html 다운로드
    html = download("get", baseURL)
    dom = BeautifulSoup(html.text, "html.parser")
    
    # 게시물 scrapping
    # 아직
    
    # 다음페이지 넘어가서 url 가져오기
    if not queue:
        current_page = int(dom.select(".page_inert")[0].text)
        next_url = requests.compat.urljoin(url, dom.select("td#page_list")[0].find_all("a", text=current_page+1)[0]["href"])
        next_html = download("get", next_url)
        next_dom = BeautifulSoup(next_html.text, "html.parser")        
        queue.extend([requests.compat.urljoin(url, tag.find_parent()["href"]) for tag in next_dom.select(".list_title")])
        
    print("Queue: {0}, Seen: {1}".format(len(queue), len(seen)))

Queue: 30, Seen: 1
Queue: 29, Seen: 2
Queue: 28, Seen: 3
Queue: 27, Seen: 4
Queue: 26, Seen: 5
Queue: 25, Seen: 6
Queue: 24, Seen: 7
Queue: 23, Seen: 8
Queue: 22, Seen: 9
Queue: 21, Seen: 10
Queue: 20, Seen: 11
Queue: 19, Seen: 12
Queue: 18, Seen: 13
Queue: 17, Seen: 14
Queue: 16, Seen: 15
Queue: 15, Seen: 16
Queue: 14, Seen: 17
Queue: 13, Seen: 18
Queue: 12, Seen: 19
Queue: 11, Seen: 20
Queue: 10, Seen: 21
Queue: 9, Seen: 22
Queue: 8, Seen: 23
Queue: 7, Seen: 24
Queue: 6, Seen: 25
Queue: 5, Seen: 26
Queue: 4, Seen: 27
Queue: 3, Seen: 28
Queue: 2, Seen: 29
Queue: 1, Seen: 30
Queue: 30, Seen: 31
Queue: 29, Seen: 32
Queue: 28, Seen: 33
Queue: 27, Seen: 34
Queue: 26, Seen: 35
Queue: 25, Seen: 36
Queue: 24, Seen: 37
Queue: 23, Seen: 38
Queue: 22, Seen: 39
Queue: 21, Seen: 40
Queue: 20, Seen: 41
Queue: 19, Seen: 42


KeyboardInterrupt: 

### 3. 스크래핑하기

In [167]:
# 게시물의 html을 가져오자

free_html = download("get", title_urls[1])
free_dom = BeautifulSoup(free_html.text, "html.parser")

In [168]:
# 제목 가져오기

free_dom.select(".view_title2")[0].text

'커플룩~'

In [169]:
free_dom.select_one(".view_title2").text

'커플룩~'

In [171]:
# 본문 가져오기
# 안됨?

free_dom.select("td.board-contents")

[]

In [172]:
# 이거도 안됨

free_dom.select("td[class^=board]")

[]

In [177]:
# 본문 가져오기
# 소스코드를 까보면, 페이지를 거지같이 만들었다는 것을 알 수 있음.
# 소스코드:   <td class='board-contents' align="left" valign=top class=han>
# td의 class를 board-contents라고 해놓고 태그 안에 han이라고 또 해놨다.
# 뒤의 class(=han)이 앞의 class(=board-contents)를 덮어썼다. 그래서 안나오는 것이다.
# 이따위로 만들어도 에러 안나고 실행되는 html이 참 신기;;

free_dom.select_one("table.pic_bg td.han").text.strip()

'얼릉 나아서 집에가자~~\n힘드네요!'

In [192]:
# 댓글 가져오기

free_dom.select(".comment_wrapper")[0].text.strip()

'0 0 \n\n애들 덕분에 피셔가셧네 ㅎㅎ\n\xa0\n\xa0\n애들 덕분에 피셔가셧네 ㅎㅎ\r\n\xa0\r\n\xa0\n\n\n2019-07-1615:46:29'

In [204]:
free_dom.select(".comment_wrapper [id^=commentContent]")

[<div class="han" id="commentContent_67878571" style="position:relative; overflow:hidden; width:100%;">애들 덕분에 피셔가셧네 ㅎㅎ
 <p> </p>
 <p> </p></div>,
 <div class="han" id="commentContent_67878578" style="position:relative; overflow:hidden; width:100%;"><p>시원하긴한데..엄청피곤해요 ㅜㅜ</p>
 <p>잠을못자서</p></div>,
 <div class="han" id="commentContent_67878714" style="position:relative; overflow:hidden; width:100%;">원숭이님도 입원하셨나요?ㅜ</div>,
 <div class="han" id="commentContent_67878817" style="position:relative; overflow:hidden; width:100%;"><p>아들.딸이요!</p></div>,
 <div class="han" id="commentContent_67878872" style="position:relative; overflow:hidden; width:100%;"><p>까치발 귀여워요~</p>
 <p>
 <br>
 </br></p></div>]

In [205]:
[comment.text.strip() for comment in free_dom.select(".comment_wrapper [id^=commentContent]")]

['애들 덕분에 피셔가셧네 ㅎㅎ',
 '시원하긴한데..엄청피곤해요 ㅜㅜ\n잠을못자서',
 '원숭이님도 입원하셨나요?ㅜ',
 '아들.딸이요!',
 '까치발 귀여워요~']

In [187]:
# id만으로도 뽑아올 수 있음.

free_dom.select("[id^=commentContent]")

[<div class="han" id="commentContent_67878571" style="position:relative; overflow:hidden; width:100%;">애들 덕분에 피셔가셧네 ㅎㅎ
 <p> </p>
 <p> </p></div>,
 <div class="han" id="commentContent_67878578" style="position:relative; overflow:hidden; width:100%;"><p>시원하긴한데..엄청피곤해요 ㅜㅜ</p>
 <p>잠을못자서</p></div>,
 <div class="han" id="commentContent_67878714" style="position:relative; overflow:hidden; width:100%;">원숭이님도 입원하셨나요?ㅜ</div>,
 <div class="han" id="commentContent_67878817" style="position:relative; overflow:hidden; width:100%;"><p>아들.딸이요!</p></div>,
 <div class="han" id="commentContent_67878872" style="position:relative; overflow:hidden; width:100%;"><p>까치발 귀여워요~</p>
 <p>
 <br>
 </br></p></div>]

In [233]:
# 이렇게도 가능함.

[comment.text.strip() for comment in free_dom.select("[id^=commentContent]")]

['애들 덕분에 피셔가셧네 ㅎㅎ',
 '시원하긴한데..엄청피곤해요 ㅜㅜ\n잠을못자서',
 '원숭이님도 입원하셨나요?ㅜ',
 '아들.딸이요!',
 '까치발 귀여워요~']

In [207]:
# 스크래핑 함수로 만들기

def scraping(dom):
    return {"title":dom.select_one(".view_title2").text.strip(),
           "content":dom.select_one("table.pic_bg td.han").text.strip(),
           "comments":[comment.text.strip() for comment in dom.select(".comment_wrapper [id^=commentContent]")]}

### 스크래핑 중간 점검

In [209]:
# 사전 준비
url = "http://www.ppomppu.co.kr/zboard/zboard.php"
params = {"id":"freeboard"}
html = download('get', url, params)
dom = BeautifulSoup(html.text, "html.parser")
title_urls = [requests.compat.urljoin(url, tag.find_parent()["href"]) for tag in dom.select(".list_title")]

In [213]:
# 가즈아~~

queue = []
queue.extend(title_urls)
seen = []
results = []

while queue:
    baseURL = queue.pop(0)
    seen.append(baseURL)
    
    # 게시물 html 다운로드
    html = download("get", baseURL)
    dom = BeautifulSoup(html.text, "html.parser")
    
    # 게시물 scrapping
    # 게시물에 댓글이 없거나 할 경우에 대비한 예외처리문
    try:
        results.append(scraping(dom))
    except AttributeError:
        print("게시물에 정보가 없습니다.")      
    
    
    # 다음페이지 넘어가서 url 가져오기
    if not queue:
        current_page = int(dom.select(".page_inert")[0].text)
        next_url = requests.compat.urljoin(url, dom.select("td#page_list")[0].find_all("a", text=current_page+1)[0]["href"])
        next_html = download("get", next_url)
        next_dom = BeautifulSoup(next_html.text, "html.parser")        
        queue.extend([requests.compat.urljoin(url, tag.find_parent()["href"]) for tag in next_dom.select(".list_title")])
        
    print("Queue: {0}, Seen: {1}".format(len(queue), len(seen)))

게시물에 정보가 없습니다.
Queue: 30, Seen: 1
Queue: 29, Seen: 2
Queue: 28, Seen: 3
Queue: 27, Seen: 4
Queue: 26, Seen: 5
Queue: 25, Seen: 6
Queue: 24, Seen: 7
Queue: 23, Seen: 8
게시물에 정보가 없습니다.
Queue: 22, Seen: 9
Queue: 21, Seen: 10
Queue: 20, Seen: 11
Queue: 19, Seen: 12


KeyboardInterrupt: 

---

### 4. DB에 연결해서 저장하기

In [194]:
import sqlite3

In [218]:
con = sqlite3.connect("ppomppu.db")

In [219]:
cur = con.cursor()

In [220]:
# 제목과 본문을 담는 테이블 1개
# 댓글을 담는 테이블 1개

cur.executescript("""
DROP TABLE IF EXISTS table3;
CREATE TABLE table3(
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    title TEXT NOT NULL,
    content TECT NOT NULL,
    date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL);
    
DROP TABLE IF EXISTS table4;
CREATE TABLE table4(
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    table3_id INEGER NOT NULL,
    comment TEXT NOT NULL);
""")

In [221]:
con.commit()

In [ ]:
# 이 코드는 실행 x
# while문에 넣기 위해서 만든 코드임.

cur.execute("""
    INSERT INTO table3 (title, content) 
    VALUE(?, ?)""", [content["title"], content["body"]])
con.commit()

cur.execute("""
    SELECT id
    FROM table3
    WHERE title=? AND content=?;
    """, [content["title"], content["body"]])
table3_id = cur.fetchone()[0]

for comment in content["commenst"]:
    cur.execute("""
        INSERT INTO table4 (table3_id, comment)
        VALUES (?, ?)
        """, [table3_id, comment])
    con.commit()

In [226]:
# 가즈아~~

queue = []
queue.extend(title_urls)
seen = []

while queue:
    baseURL = queue.pop(0)
    seen.append(baseURL)
    
    # 게시물 html 다운로드
    html = download("get", baseURL)
    dom = BeautifulSoup(html.text, "html.parser")
    
    # 게시물 scrapping
    # 게시물에 댓글이 없거나 할 경우에 에러발생을 대비한 예외처리문
    try:
        content = scraping(dom)
    except AttributeError
    else: 
        cur.execute("""
            INSERT INTO table3 (title, content) 
            VALUES(?, ?)""", [content["title"], content["content"]])
        con.commit()

        cur.execute("""
            SELECT id
            FROM table3
            WHERE title=? AND content=?;
            """, [content["title"], content["content"]])
        table3_id = cur.fetchone()[0]

        for comment in content["comments"]:
            cur.execute("""
                INSERT INTO table4 (table3_id, comment)
                VALUES (?, ?)
                """, [table3_id, comment])
            con.commit()
    
    # 다음페이지 넘어가서 url 가져오기
    if not queue:
        current_page = int(dom.select(".page_inert")[0].text)
        next_url = requests.compat.urljoin(url, dom.select("td#page_list")[0].find_all("a", text=current_page+1)[0]["href"])
        next_html = download("get", next_url)
        next_dom = BeautifulSoup(next_html.text, "html.parser")        
        queue.extend([requests.compat.urljoin(url, tag.find_parent()["href"]) for tag in next_dom.select(".list_title")])
        
    print("Queue: {0}, Seen: {1}".format(len(queue), len(seen)))

게시물에 정보가 없습니다.
Queue: 30, Seen: 1
Queue: 29, Seen: 2
Queue: 28, Seen: 3
Queue: 27, Seen: 4
Queue: 26, Seen: 5
Queue: 25, Seen: 6
Queue: 24, Seen: 7
Queue: 23, Seen: 8
게시물에 정보가 없습니다.
Queue: 22, Seen: 9
Queue: 21, Seen: 10
Queue: 20, Seen: 11
Queue: 19, Seen: 12
Queue: 18, Seen: 13
Queue: 17, Seen: 14
Queue: 16, Seen: 15
Queue: 15, Seen: 16
Queue: 14, Seen: 17
Queue: 13, Seen: 18
Queue: 12, Seen: 19
Queue: 11, Seen: 20
Queue: 10, Seen: 21
Queue: 9, Seen: 22
Queue: 8, Seen: 23
Queue: 7, Seen: 24
Queue: 6, Seen: 25
Queue: 5, Seen: 26
Queue: 4, Seen: 27
Queue: 3, Seen: 28
Queue: 2, Seen: 29
Queue: 1, Seen: 30
Queue: 30, Seen: 31
Queue: 29, Seen: 32
Queue: 28, Seen: 33
Queue: 27, Seen: 34
Queue: 26, Seen: 35
Queue: 25, Seen: 36
Queue: 24, Seen: 37
Queue: 23, Seen: 38
Queue: 22, Seen: 39
Queue: 21, Seen: 40
Queue: 20, Seen: 41
Queue: 19, Seen: 42
Queue: 18, Seen: 43
Queue: 17, Seen: 44
Queue: 16, Seen: 45
Queue: 15, Seen: 46
Queue: 14, Seen: 47
Queue: 13, Seen: 48
Queue: 12, Seen: 49
Queue: 1

InvalidSchema: No connection adapters were found for 'javascript:go_page( 'freeboard', 6543036, 2 )'